# Warm up

The code below showcases a convolutional network in Keras. It was designed to classify 100x100 rgb images into 10 classes.
This network... quite frankly, it sucks. Can you guess what's the problem? Is there just one problem?

In [5]:
import keras
import keras.layers as L
import keras.initializers as init

In [4]:
net = keras.models.Sequential()

net.add(L.InputLayer([100, 100, 3]))

net.add(L.Conv2D(filters=512, kernel_size=(3, 3), 
                 kernel_initializer=init.zeros()))
net.add(L.Activation('relu'))

net.add(L.Conv2D(filters=128, kernel_size=(3, 3), 
                 kernel_initializer=init.zeros()))
net.add(L.Activation('relu'))

net.add(L.Conv2D(filters=32, kernel_size=(3, 3), 
                 kernel_initializer=init.zeros()))
net.add(L.Activation('relu'))

net.add(L.MaxPool2D(pool_size=(6, 6)))

net.add(L.Conv2D(filters=8, kernel_size=(10, 10), 
                 kernel_initializer=init.RandomNormal(), padding='same'))
net.add(L.Activation('relu'))


net.add(L.Conv2D(filters=8, kernel_size=(10, 10), 
                 kernel_initializer=init.RandomNormal(), padding='same'))
net.add(L.Activation('relu'))

net.add(L.MaxPool2D(pool_size=(3, 3)))

net.add(L.Flatten()) # convert 3d tensor to a vector of features

net.add(L.Dense(units=512))
net.add(L.Activation('softmax'))

net.add(L.Dropout(rate=0.5))

net.add(L.Dense(units=512))
net.add(L.Activation('softmax'))

net.add(L.Dense(units=10))
net.add(L.Activation('sigmoid'))
net.add(L.Dropout(rate=0.5))

* [Conv2D](https://keras.io/layers/convolutional/#conv2d) - performs convolution:
    * filters: number of output channels;
    * kernel_size: an integer or tuple/list of 2 integers, specifying the width and height of the 2D convolution window;
    * padding: padding="same" adds zero padding to the input, so that the output has the same width and height, padding='valid' performs convolution only in locations where kernel and the input fully overlap;
    * activation: "relu", "tanh", etc.
    * input_shape: shape of input.
* [MaxPooling2D](https://keras.io/layers/pooling/#maxpooling2d) - performs 2D max pooling.
* [Flatten](https://keras.io/layers/core/#flatten) - flattens the input, does not affect the batch size.
* [Dense](https://keras.io/layers/core/#dense) - fully-connected layer.
    * Activation - applies an activation function.
* [LeakyReLU](https://keras.io/layers/advanced-activations/#leakyrelu) - applies leaky relu activation.
* [Dropout](https://keras.io/layers/core/#dropout) - applies dropout.

## Book of grudges
* zero init for weights will cause symmetry effect
* Too many filters for first 3x3 convolution - will lead to enormous matrix while there's just not enough relevant combinations of 3x3 images (overkill).
* Usually the further you go, the more filters you need.
* large filters (10x10 is generally a bad pactice, and you definitely need more than 10 of them
* the second of 10x10 convolution gets 8x6x6 image as input, so it's technically unable to perform such convolution.
* Softmax nonlinearity effectively makes only 1 or a few neurons from the entire layer to "fire", rendering 512-neuron layer almost useless. Softmax at the output layer is okay though
* Dropout after probability prediciton is just lame. A few random classes get probability of 0, so your probabilities no longer sum to 1 and crossentropy goes -inf.

In this exercise you have to train a new Convolutional Neural Network from scratch for the classification of images.

1. For this we will use the Keras library.
2. The aim is to achieve 99% accuracy (on validation/test set) the MNIST dataset http://yann.lecun.com/exdb/mnist/.
3. We have provided a basic Keras implementation of a CNN.
4. You are allowed to do whatever you want (except copy pasting) with the network as long as it is explained in your report.
5. Feel free to change the architecture of the network as well as parameters (e.g. learning rate, kernel sizes, ...).
6. You can try to guess parameters manually of you want, just make sure that it performs better than 99% on the validation set.
7. Sketch the final network architecture in your report.
8. Make sure you train the network on the GPU, otherwise it will be too slow.
9. Explain the plots: learning curve, accuracy wrt epoch.

# Going bigger

* Use `tf.keras.datasets.cifar10.load_data()` to get the data
* split to 70 - 30 train / val using `train_test_split`
* normalize the input like $x_{\text{norm}} = \frac{x}{255} - 0.5$
* We need to convert class labels to one-hot encoded vectors. Use `keras.utils.to_categorical`.

In [1]:
# normalize inputs
# convert class labels to one-hot encoded, should have shape (?, NUM_CLASSES)
import tensorflow as tf
train, test = tf.keras.datasets.mnist.load_data()
train[0].shape, train[1].shape


((60000, 28, 28), (60000,))

In [2]:
X_train_origin, y_train_origin = train[0], train[1]
X_test_origin, y_test_origin = test[0], test[1]
X_test_origin.shape

(10000, 28, 28)

In [3]:
X_train_norm, X_test_norm = X_train_origin/255, X_test_origin/255    

In [8]:
from keras.layers import BatchNormalization, Dropout, Dense, Conv2D, Flatten, Activation
net = keras.models.Sequential()
net.add(Conv2D(filters=32, kernel_size = 5, strides = 1, padding = "same"))
net.add(BatchNormalization())
net.add(Activation('relu'))  
net.add(Conv2D(filters=64, kernel_size = (3,3), strides = 1, padding = "same"))  
net.add(BatchNormalization())
net.add(Activation('relu'))

net.add(Conv2D(filters=128, kernel_size = (3,3), strides = 1, padding = "same"))
net.add(BatchNormalization())
net.add(Activation('relu'))  

net.add(Flatten())
net.add(Dense(100, 'relu'))
net.add(Dropout(0.1))
net.add(Dense(10, 'softmax'))

In [7]:
import numpy as np
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
net.compile("adam", loss, ["accuracy"])
net.fit(np.expand_dims(X_train_norm, -1), y_train_origin, 124, 10, verbose=2, validation_split = .1)

Epoch 1/10
436/436 - 224s - loss: 1.1095 - accuracy: 0.7797 - val_loss: 0.2304 - val_accuracy: 0.9527
Epoch 2/10
436/436 - 231s - loss: 0.2997 - accuracy: 0.9015 - val_loss: 0.1096 - val_accuracy: 0.9737
Epoch 3/10
436/436 - 239s - loss: 0.2447 - accuracy: 0.9137 - val_loss: 0.1114 - val_accuracy: 0.9667
Epoch 4/10
436/436 - 241s - loss: 0.2071 - accuracy: 0.9273 - val_loss: 0.1431 - val_accuracy: 0.9702
Epoch 5/10
436/436 - 247s - loss: 0.1892 - accuracy: 0.9379 - val_loss: 0.0781 - val_accuracy: 0.9807
Epoch 6/10
436/436 - 244s - loss: 0.1628 - accuracy: 0.9479 - val_loss: 0.0744 - val_accuracy: 0.9818
Epoch 7/10
436/436 - 240s - loss: 0.1398 - accuracy: 0.9551 - val_loss: 0.0672 - val_accuracy: 0.9837
Epoch 8/10
436/436 - 240s - loss: 0.1302 - accuracy: 0.9584 - val_loss: 0.0979 - val_accuracy: 0.9793
Epoch 9/10
436/436 - 254s - loss: 0.1226 - accuracy: 0.9606 - val_loss: 0.0564 - val_accuracy: 0.9892
Epoch 10/10
436/436 - 247s - loss: 0.1085 - accuracy: 0.9660 - val_loss: 0.0461 - 

In [11]:
net.evaluate(np.expand_dims(X_test_norm, -1), y_test_origin)

313/313 [==============================] - 12s 37ms/step - loss: 2.0122 - accuracy: 0.7517


[2.012183666229248, 0.7516999840736389]